<a href="https://colab.research.google.com/github/MatthewYancey/GANime/blob/master/src/inference_lama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lama Inference
This notebook takes a given lama model and runs inference on images

In [ ]:
print('\n> Cloning the repo')
!git clone https://github.com/saic-mdal/lama.git

print('\n> Install dependencies')
!pip install -r lama/requirements.txt --quiet
!pip install wget --quiet

print('\n> Changing the dir to:')
% cd /content/lama

print('\n> Download the model')
!curl -L $(yadisk-direct https://disk.yandex.ru/d/EgqaSnLohjuzAg) -o lama-models.zip
!unzip lama-models.zip

print('>fixing opencv')
!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet


print('\n> Init mask-drawing code')
import base64, os
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
import wget
from shutil import copyfile
import shutil

print('Downloading test data')
!curl -L $(yadisk-direct https://disk.yandex.ru/d/xKQJZeVRk5vLlQ) -o LaMa_test_images.zip
!unzip LaMa_test_images.zip

In [ ]:
import glob
import cv2

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
MODEL_PATH = '/content/gdrive/MyDrive/repos/GANime/data_out/lama_model/custom_lama_v2'

# INPUT_ZIP = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/test_masks.zip'
# INPUT_ZIP = '/content/gdrive/MyDrive/repos/GANime/data_out/example_output/pokemon.zip'
INPUT_ZIP = '/content/gdrive/MyDrive/repos/GANime/data_out/example_output/wizard_of_oz.zip'
# INPUT_ZIP = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/test_masks.zip'
INPUT_DIR = '/content/input/'

OUTPUT_DIR = '/content/lama/output/'
OUTPUT_ZIP = '/content/gdrive/MyDrive/repos/GANime/data_out/example_output_model/wizard_of_oz/lama' # leave off the extention

In [ ]:
if os.path.isdir(INPUT_DIR):
    shutil.rmtree(INPUT_DIR)

shutil.unpack_archive(INPUT_ZIP, INPUT_DIR)

In [ ]:
# produces the masks
def prep_data(in_dir, out_dir, img_width, img_height, single_side):
    # # creates a test set of validaiton
    # if os.path.isdir(out_dir):
    #     shutil.rmtree(out_dir)
    # os.mkdir(out_dir)

    img_paths = glob.glob(in_dir + '*')

    # makes a mask and saves it to correspond with the names of images
    inside = np.zeros((img_height, img_width - 2*single_side, 3), np.uint8)
    outsides = np.ones((img_height, single_side, 3), np.uint8) * 255
    mask = cv2.hconcat([outsides, inside, outsides])

    for img_path in img_paths:
        # converts image
        img = cv2.imread(img_path)
        new_img_path = out_dir + os.path.basename(img_path).replace('.jpg', '.png')
        cv2.imwrite(new_img_path, img)

        # saves mask
        mask_name = img_path.replace('.jpg', '') + '_mask.png'
        cv2.imwrite(mask_name, mask)

        os.remove(img_path)

prep_data(INPUT_DIR, INPUT_DIR, 512, 288, 64)

In [ ]:
# Runs inference on the lama model
if os.path.isdir(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
print('Predicting')
!export TORCH_HOME=$(pwd) && export PYTHONPATH=. && python3 bin/predict.py model.path=/content/gdrive/MyDrive/repos/GANime/data_out/lama_model/custom_lama indir=/content/input/ outdir=$(pwd)/output
# !export TORCH_HOME=$(pwd) && export PYTHONPATH=. && python3 bin/predict.py model.path={MODEL_PATH} indir=/content/input/ outdir=/content/lama/output/

In [ ]:
shutil.make_archive(OUTPUT_ZIP, 'zip', OUTPUT_DIR)